In [1]:
library(AppliedPredictiveModeling)
data(concrete)

Warning message:
“package ‘AppliedPredictiveModeling’ was built under R version 3.4.4”

In [4]:
library(plyr)
averaged <- ddply(mixtures,.(Cement,BlastFurnaceSlag,FlyAsh,Water,
                            Superplasticizer,CoarseAggregate,FineAggregate,Age),
                 function(x) c(CompressiveStrength=mean(x$CompressiveStrength)))

In [5]:
library(caret)
set.seed(830)
trn_id <- createDataPartition(averaged$CompressiveStrength,p=3/4,list=F)
trn <- averaged[trn_id,]
vld <- averaged[-trn_id,]

In [3]:
library(parallel)
setDefaultCluster(makeCluster(4))

In [8]:
set.seed(830)
ctrl <- trainControl(method = "repeatedcv",repeats = 5,number=10)
grid_cubist <- expand.grid(
                           committees=c(1,5,10,50,75,100),
                           neighbors=c(0,1,3,5,7,9)
                          )
fit_cubist <- train(CompressiveStrength~.,data=trn,method = "cubist",
                   tuneGrid=grid_cubist,trControl=ctrl)


In [9]:
fit_cubist$bestTune

,committees,neighbors
33,100,3


In [14]:
### Create a function to maximize compressive strength* while keeping
### the predictor values as mixtures. Water (in x[7]) is used as the 
### 'slack variable'. 
modelPrediction <- function(x, mod, limit = 2500)
{
  if(x[1] < 0 | x[1] > 1) return(10^38)
  if(x[2] < 0 | x[2] > 1) return(10^38)
  if(x[3] < 0 | x[3] > 1) return(10^38)
  if(x[4] < 0 | x[4] > 1) return(10^38)
  if(x[5] < 0 | x[5] > 1) return(10^38)
  if(x[6] < 0 | x[6] > 1) return(10^38)
  
  x <- c(x, 1 - sum(x))
  
  if(x[7] < 0.05) return(10^38)
  
  tmp <- as.data.frame(t(x))
  names(tmp) <- c('Cement','BlastFurnaceSlag','FlyAsh',
                  'Superplasticizer','CoarseAggregate',
                  'FineAggregate', 'Water')
  tmp$Age <- 28
  -predict(mod, tmp)
}

In [12]:
head(trn)

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,CompressiveStrength
1,0.22309440,0.00000000,0,0.06692832,0.001032844,0.4296633,0.2792811,28,79.99
2,0.22172039,0.00000000,0,0.06651612,0.001026483,0.4331759,0.2775611,28,61.89
3,0.14917003,0.06393001,0,0.10228802,0.000000000,0.4181247,0.2664872,270,40.27
5,0.08534961,0.05689974,0,0.08251322,0.000000000,0.4204736,0.3547638,360,44.30
6,0.12036199,0.05158371,0,0.10316742,0.000000000,0.4217195,0.3031674,90,47.03
7,0.17048004,0.04262001,0,0.10228802,0.000000000,0.4181247,0.2664872,365,43.70


In [13]:
### Get mixtures at 28 days 
trn_sub <- subset(trn, Age == 28)

### Center and scale the data to use dissimilarity sampling
trn_scaled <- predict(
    preProcess(trn_sub[, -(8:9)], c("center", "scale")), 
    trn_sub[, 1:7])

In [ ]:
### Randomly select a few mixtures as a starting pool

set.seed(91)
startMixture <- sample(1:nrow(subTrain), 1)
starters <- scaledTrain[startMixture, 1:7]
pool <- scaledTrain
index <- maxDissim(starters, pool, 14)
startPoints <- c(startMixture, index)

starters <- subTrain[startPoints,1:7]
startingValues <- starters[, -4]

### For each starting mixture, optimize the Cubist model using
### a simplex search routine

cbResults <- startingValues
cbResults$Water <- NA
cbResults$Prediction <- NA

for(i in 1:nrow(cbResults))
{
  results <- optim(unlist(cbResults[i,1:6]),
                   modelPrediction,
                   method = "Nelder-Mead",
                   control=list(maxit=5000),
                   mod = cbFit)
  cbResults$Prediction[i] <- -results$value
  cbResults[i,1:6] <- results$par
}
cbResults$Water <- 1 - apply(cbResults[,1:6], 1, sum)
cbResults <- subset(cbResults, Prediction > 0 & Water > .02)
cbResults <- cbResults[order(-cbResults$Prediction),][1:3,]
cbResults$Model <- "Cubist"
